In [40]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical
from copy import deepcopy

In [41]:
import import_ipynb
import pandas as pd
import numpy as np

In [42]:
import Model 
from Model import *

In [43]:
import utils
from utils import *

In [44]:
import ewc
from ewc import *

In [45]:
def train_loop():
    return 0

In [ ]:

def train_loop(model, OPTIMIZER, MAX_LABEL, data, test_size,
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    
    weights = model.trainable_weights
    fisher_matrix = [tf.zeros_like(w) for w in weights]

    output = []

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) < MAX_LABEL:
            
            if i == 0:
                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = to_input(train, MAX_LABEL)

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=2)

                test_seq, test_label = to_input(test, MAX_LABEL)
                loss, accuracy = model.evaluate(test_seq, test_label, batch_size=32, verbose=1)
                print(f"Task_0 training accuracy: {accuracy:.4f}")
                output.append(f'task_{i} accuracy : {accuracy:.4f}')

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)
                
                train_seq, train_label = to_input(train, MAX_LABEL)

                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=2)

                inc_test_seq, inc_test_label = to_input(inc_test, MAX_LABEL)
                loss, inc_accuracy = model.evaluate(inc_test_seq, inc_test_label, batch_size=32, verbose=1)

                print(f"Task_{i} accuracy: {inc_accuracy:.4f}")


                test_seq, test_label = to_input(test, MAX_LABEL)

                loss, accuracy = model.evaluate(test_seq, test_label, batch_size=32, verbose=1)
                print(f"Task ~{i-1} accuracy after training on Task_{i}: {accuracy:.4f}")
                output.append(f'task_{i-1} accuracy on task_{i}: {accuracy:.4f}')
                
                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, inc_test_seq, num_sample=len(inc_test_seq))

                i = i + 1

        else:
            print(output)
            break 

In [49]:

def train_nonbase_AB(model, OPTIMIZER, data, test_size, task_A, task_B,
                      MAX_LABEL, epochs_A, epochs_B):

    model.compile(loss=CategoricalCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])


    ###### task A
    part_A = split_by_label(data, 0, task_A)
    train_A, test_A = split_train_test(part_A, test_size=test_size, random_state=11)

    train_A_seq, train_A_label = to_input(train_A, MAX_LABEL = MAX_LABEL)

    history = model.fit(x=train_A_seq, y=train_A_label, epochs=epochs_A, verbose=2)

    test_A_seq, test_A_label = to_input(test_A, MAX_LABEL = MAX_LABEL)
    loss, accuracy = model.evaluate(test_A_seq, test_A_label, batch_size=32, verbose=1)

    print(f"\ntask A Test Accuracy: {accuracy:.4f}\n")

    ###### task B
    part_B = split_by_label(data, task_A+1, task_A+task_B)
    train_B, test_B = split_train_test(part_B, test_size=test_size, random_state=11)

    train_B_seq, train_B_label = to_input(train_B, MAX_LABEL = MAX_LABEL)

    history = model.fit(x=train_B_seq, y=train_B_label, epochs=epochs_B, verbose=2)

    test_B_seq, test_B_label = to_input(test_B, MAX_LABEL = MAX_LABEL)
    loss, accuracy = model.evaluate(test_B_seq, test_B_label, batch_size=32, verbose=1)

    print(f"\ntask B Accuracy: {accuracy:.4f}\n")


    loss, accuracy = model.evaluate(test_A_seq, test_A_label, batch_size=32, verbose=1)
    print(f"\nforgetted Accuracy: {accuracy:.4f}\n")

In [ ]:

def train_loop_with_val(model, OPTIMIZER, MAX_LABEL, data, test_size, 
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    
    weights = model.trainable_weights
    fisher_matrix = [tf.zeros_like(w) for w in weights]

    output = []

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) < MAX_LABEL:
            
            if i == 0:
                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = to_input(train, MAX_LABEL)
                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=2)

                test_seq, test_label = to_input(test, MAX_LABEL)
                loss, accuracy = model.evaluate(test_seq, test_label, batch_size=32, verbose=1)

                print(f"Task_0 training accuracy: {accuracy:.4f}")
                output.append(f'task_{i} accuracy : {accuracy:.4f}')

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)
                
                train_seq, train_label = to_input(train, MAX_LABEL)

                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=2)

                inc_test_seq, inc_test_label = to_input(inc_test, MAX_LABEL)
                loss, inc_accuracy = model.evaluate(inc_test_seq, inc_test_label, batch_size=32, verbose=1)

                print(f"Task_{i} accuracy: {inc_accuracy:.4f}")


                test_seq, test_label = to_input(test, MAX_LABEL)

                loss, accuracy = model.evaluate(test_seq, test_label, batch_size=32, verbose=1)
                print(f"Task ~{i-1} accuracy after training on Task_{i}: {accuracy:.4f}")
                output.append(f'task_{i-1} accuracy on task_{i}: {accuracy:.4f}')
                
                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, inc_test_seq, num_sample=len(inc_test_seq))

                i = i + 1

        else:
            print(output)
            break 